In [1]:
!pip install requests beautifulsoup4 python-dotenv openai


In [2]:

import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from pathlib import Path
from openai import OpenAI


In [3]:
# Load the .env file from the parent folder
env_path = Path("..") / ".env"
load_dotenv(dotenv_path=env_path)

api_key = os.getenv("OPENAI_API_KEY")
print(f"✅ Loaded API key: {api_key[:8]}..." if api_key else "❌ API key not loaded.")


✅ Loaded API key: sk-proj-...


In [4]:
def fetch_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    article_text = ' '.join([p.get_text() for p in paragraphs])
    return article_text


In [5]:
client = OpenAI(api_key=api_key)

def summarize_text(text, max_tokens=300):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes the most important articles on the webpage."},
            {"role": "user", "content": f"Please summarize the following website using Markdown with sections like **Summary**, **Key Points**, and **Conclusion**:\n\n{text}"}
        ],
        temperature=0.7,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content


In [6]:
def summarize_url(url):
    print(f"📥 Fetching article from: {url}")
    article = fetch_article(url)
    
    if not article or len(article.strip()) < 200:
        return "⚠️ Article is too short or could not be extracted."

    print("🤖 Generating summary...")
    summary = summarize_text(article)
    return summary


In [7]:
from IPython.display import Markdown, display

url = "https://foxnews.com"
summary = summarize_url(url)

print("✅ Summary generated. Displaying below:")
display(Markdown(summary))


📥 Fetching article from: https://foxnews.com
🤖 Generating summary...
✅ Summary generated. Displaying below:


**Summary:**
The webpage provides information about various channels and services offered by FOX, such as Fox News Channel, Fox Business Channel, Fox Nation, FOX Weather, Fox Radio, and OutKick. Some services require a pay TV login, while others are free to access.

**Key Points:**
- FOX offers a range of channels and services including news, business, weather, radio, and sports content.
- Fox News Channel and Fox Business Channel require a pay TV login to access.
- Fox Nation and OutKick may require subscriptions but FOX Weather and Fox Radio are free to watch and listen, respectively.
- The material on the website is copyrighted and may not be published, broadcast, rewritten, or redistributed without permission.
- Market data is provided by Factset, and mutual fund and ETF data are provided by Refinitiv Lipper.
- The webpage is powered and implemented by FactSet Digital Solutions.

**Conclusion:**
The webpage serves as a hub for accessing various FOX channels and services, catering to different preferences such as news, business, weather, radio, and sports. Users are informed about the access requirements for each service and the copyright restrictions in place. Market data sources are also disclosed, adding credibility to the information provided on the site.